# Python RAG Pattern with Semantic Kernel and PgVector

## Azure PostgreSQL Flexible Server - PGVector Setup in Azure

Manual Instructions:

- Create a Flexible server instance in the Azure Portal
- After creation, navigate to the Server Parameters pane:
  - Search for azure.extensions
  - Check the `Vector` value
  - Save the changes and wait for the server to deploy
- After deployment, open the instance and navigate to the `Database` panel:
  - Click `Connect` link on the Postgres database
    - Using the Cloud Shell psql, active the vector extension by typing: `CREATE EXTENSION vector;`

Useful commands:

- `truncate table public."PYCollection";`

## Setup

### Load required packages

In [10]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)
from semantic_kernel.connectors.memory.postgres.postgres_memory_store import (
    PostgresMemoryStore,
)
from dotenv import load_dotenv
import os

COLLECTION_NAME = "PYCollection"

### Load the environment variables

In [11]:
load_dotenv()
endpoint = os.getenv("GPT_OPENAI_ENDPOINT")
api_key = os.getenv("GPT_OPENAI_KEY")
gpt_deployment_name = os.getenv("GPT_OPENAI_DEPLOYMENT_NAME")
conn_str = os.getenv("PG_CONN_STR_PY")
ada_deployment_name = "ada"

### Get a kernel instance configured for text completions and embeddings

In [12]:
kernel = sk.Kernel()
azure_chat_service = AzureChatCompletion(deployment_name=gpt_deployment_name, endpoint=endpoint, api_key=api_key)
azure_text_embedding = AzureTextEmbedding(deployment_name=ada_deployment_name, endpoint=endpoint, api_key=api_key)
kernel.add_chat_service("chat_completion", azure_chat_service)
kernel.add_text_embedding_generation_service("ada", azure_text_embedding)

mem_store = PostgresMemoryStore(conn_str,1536,1,3)
kernel.register_memory_store(memory_store=mem_store)
kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "text_memory")
print("Kernel is ready to use")

Kernel is ready to use


## Ingestion

### Read the files and chunk them by paragraph

In [13]:
def read_file(file: str)->str:
    with open(file, "r") as f:
        return f.read()
    
def ingest_content(path:str):
    import os
    chunks = []
    files = os.listdir(path)
    for f in files:
        if f.endswith("water.txt"):            
            content = read_file("data/"+f)
            paragraphs = content.split("\n\n")
            l = len(paragraphs)
            id = 1
            for p in paragraphs:
                lid = f"{f}-{l}-{id}"
                c = {"id":lid,"chunk":p,"file":f}
                chunks.append(c)
                id += 1
    return chunks

chunks = ingest_content("data")

## Grounding

### Retrieve the memory based on a query

In [14]:
async def populate_memory(kernel: sk.Kernel, chunks: list) -> None:
    for chunk in chunks:
        await kernel.memory.save_information(collection=COLLECTION_NAME, id=chunk["id"], text=chunk["chunk"], description=chunk["file"])

await populate_memory(kernel, chunks)

### Find memories based on query, and collect the text in the memories to augment the prompt

In [15]:
async def search_memory_examples(kernel: sk.Kernel, question: str, limit: int=3, relevance=0.75) -> list:
    results = await kernel.memory.search(COLLECTION_NAME, question,limit,relevance)
    return results



## Get the context from memory

In [16]:
question = "What is the chemical composition of water?"
results = await search_memory_examples(kernel, question)
prompt_context = "Context: \"\"\"\n"

for result in results:
    prompt_context += f"Text:\n{result.text}\nSource:\n{result.description}\n"
    
prompt_context += "\"\"\""

## Process Prompt & Completion

### Create a SK function

In [17]:
promptTemplate = "{{$input}}\n\nContext: ===\n{{$context}}\n===\nAdd a source reference to the end of each sentence. e.g. Apple is a fruit [reference1.pdf][reference2.pdf]. Use only the provided text."
rag_function = kernel.create_semantic_function(prompt_template=promptTemplate, max_tokens=500, temperature=0.3)

skf_context = kernel.create_new_context()
skf_context["input"] = question
skf_context["context"] = prompt_context

### Submit the Prompt and print the results

In [18]:
result = await rag_function(context=skf_context)
print(f"user:\n{question}\n\nassistant:\n{result}")


user:
What is the chemical composition of water?

assistant:
Water is an inorganic compound with the chemical formula H2O. [wikipedia-water.txt]

It is a transparent, tasteless, odorless,[c] and nearly colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living organisms (in which it acts as a solvent[19]). [wikipedia-water.txt]

Water, a substance composed of the chemical elements hydrogen and oxygen and existing in gaseous, liquid, and solid states. [brittanica-water.txt]

Although the molecules of water are simple in structure (H2O), the physical and chemical properties of the compound are extraordinarily complicated, and they are not typical of most substances found on Earth. [brittanica-water.txt]

It is one of the most plentiful and essential of compounds. [brittanica-water.txt]

A tasteless and odourless liquid at room temperature, it has the important ability to dissolve many other substances. [brittanica-water.txt]

In